In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
include("ETO.jl")
include("RO.jl")
include("Data.jl")
include("Estimate.jl")
include("Performance.jl")
Random.seed!(2)

TaskLocalRNG()

In [2]:
N_u = 1
N = 2
S = 10000
S_test = 10000
K = 10   # 每个产品的选择项数量

max_offdiag = 1
offdiag_sign = "zero"

A_true, B_true = Generate_Coef(N_u, N,max_offdiag,offdiag_sign);

U_train, P_train = Generate_Feat_Data(N_u, N, S);
U_test, P_test = Generate_Feat_Data(N_u, N, S_test);

In [ ]:
A_hat, B_hat = Estimate_MNL_Para(U_train, P_train, S, N, N_u, N, A_true, B_true)

a_hat:
[0.005186436107860032; 0.6912277990831004;;]
b_hat:
[-0.9601403832518198 0.08542222322210638; 0.012347260784333005 -1.1561150584466964]


([0.005186436107860032; 0.6912277990831004;;], [-0.9601403832518198 0.08542222322210638; 0.012347260784333005 -1.1561150584466964])

In [4]:
a_hat

2×1 Matrix{Float64}:
 0.005186436107860032
 0.6912277990831004

In [5]:
A_true

2×1 Matrix{Float64}:
 0.0022505868897625403
 0.6807475064887184

In [6]:
b_hat

2×2 Matrix{Float64}:
 -0.96014     0.0854222
  0.0123473  -1.15612

In [7]:
B_true

2×2 Matrix{Float64}:
 -0.855881   0.0
  0.0       -1.17778

In [ ]:
function compute_prob(S,N,U_train,P_train,a_star,b_star)
    for s in 1:S
        u_this = U_train[s,:]
        p_this = P_train[s,:]
        logits = [dot(a_star[n,:], u_this) + dot(b_star[n,:], p_this) for n in 1:N]
        exp_logits = exp.(logits)
        denom = 1 + sum(exp_logits)
        probs[s, :] = exp_logits ./ denom
    end
    return probs
end

In [ ]:
probs = compute_prob(S,N,U_train,P_train,A_true,B_true);

In [ ]:
function Calculate_Choice(S,N,probs)
    choices = Vector{Int}(undef, S)
    for s in 1:S
        p = probs[s, :]
        baseline_prob = 1 - sum(p)
        full_p = vcat(baseline_prob, p)  # 添加 baseline 的概率
        choices[s] = sample(0:N, Weights(full_p))  # 随机选择（含baseline）
    end
    return choices
end 

In [ ]:
choices = Calculate_Choice(S,N,probs)
# 打印部分结果
df = DataFrame(choice = choices)
println(first(df, 20))

In [ ]:
probs[10:20, :]

In [ ]:
s = 1
p = probs[s, :]
p[1] = 0.01
p[2] = 0.98
baseline_prob = 1 - sum(p)
full_p = vcat(baseline_prob, p)  # 添加 baseline 的概率
choices[s] = sample(0:N, Weights(full_p))  # 随机选择（含baseline）
sample(0:N, Weights(full_p))

In [ ]:
p[1] = 0.01

In [ ]:
full_p

### Parameters

In [ ]:
function generate_strictly_row_diagonally_dominant(n::Int, max_offdiag,offdiag_sign)
    Mat = zeros(n, n)
    for i in 1:n
        # 在非对角线上生成随机数 [-max_offdiag, max_offdiag]
        off_diag = rand(n) .* (2max_offdiag) .- max_offdiag
        off_diag[i] = 0  # 避免给自己赋值两次

        # 计算非对角元素绝对值之和
        sum_offdiag = sum(abs, off_diag)

        # 设置对角元，使其严格大于其他元素之和
        diag_value = sum_offdiag + rand() * max_offdiag + 1e-3  # 加小量保证严格性

        if offdiag_sign == "positive" 
            Mat[i, :] = abs.(off_diag)
        end
        if offdiag_sign == "negative" 
            Mat[i, :] = -abs.(off_diag)
        end
        if offdiag_sign == "mix" 
            Mat[i, :] = off_diag
        end
        if offdiag_sign == "zero" 
            Mat[i, :] = off_diag .* 0.0
        end        
        Mat[i, i] = -abs.(diag_value)
    end
    return Mat
end

In [ ]:
function compute_oof(X_given, A, B, u, p_dag)
    price = sum(X_given .* p_dag,dims=2)[:,1]
    utilities = ones(N+1)
    utilities[1:N] = exp.(vec(A * u + B * price))

    prob = zeros(N)
    rev = zeros(N)
    for i in 1:N
        prob[i] = utilities[i] / sum(utilities)
        rev[i] = prob[i] * price[i]
    end
    # println("prob = ", prob)
    # println("rev = ", rev)
    # println("total rev = ", sum(rev))
    return sum(rev),price
end

### Iteratively search for the optimal solution

In [ ]:
# rev_opt = 0
# price_opt = zeros(N)

# rev_min = 1000
# price_min = zeros(N)
# for i in 1:K
#     p1 = p_dag[1,i]
#     for j in 1:K
#         p2 = p_dag[2,j]
#         for l in 1:K
#             p3 = p_dag[3,l]
#             price = [p1,p2,p3]
#             utilities = ones(N+1)
#             utilities[1:N] = exp.(vec(A * u + B * price))
        
#             prob = zeros(N)
#             rev = zeros(N)
#             for i in 1:N
#                 prob[i] = utilities[i] / sum(utilities)
#                 rev[i] = prob[i] * price[i]
#             end
#             if sum(rev) > rev_opt
#                 rev_opt = sum(rev)
#                 price_opt = price 
#             end

#             if sum(rev) < rev_min
#                 rev_min = sum(rev)
#                 price_min = price
#             end
#         end
#     end
# end
# println("Optimal revenue = ", rev_opt)
# println("Optimal price = ", price_opt)

# println("Minimum revenue = ", rev_min) 
# println("Minimum price = ", price_min)

In [ ]:
function Solve_RO_Price(N,N_u,K,A_lb,A_ub,B_lb,B_ub,u,p_dag,psi_lb,psi_ub,phi_lb,phi_ub)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, delta)                           # 标量 δ
    @variable(model, omega_lb[1:N,1:N_u] >= 0)            
    @variable(model, omega_ub[1:N,1:N_u] >= 0)           
    @variable(model, pi_lb[1:N,1:N] >= 0)     
    @variable(model, pi_ub[1:N,1:N] >= 0)     
    @variable(model, eta_lb[1:N,1:N_u] >= 0)            
    @variable(model, eta_ub[1:N,1:N_u] >= 0)           
    @variable(model, ups_lb[1:N,1:N] >= 0)     
    @variable(model, ups_ub[1:N,1:N] >= 0) 
    # exponential variables
    @variable(model, psi_1[1:N])                   
    @variable(model, psi_2[1:N])                   
    @variable(model, psi_3[1:N])        
    @variable(model, phi_1[1:N])                   
    @variable(model, phi_2[1:N])                   
    @variable(model, phi_3[1:N])         

    @variable(model, X[1:N, 1:K], Bin)        # 二进制变量 x_{jk}
    @variable(model, Y[1:N,1:N,1:K] <= 0)    
    @variable(model, Z[1:N,1:N,1:K] <= 0)    

    for n in 1:N
        @constraint(model, omega_lb[n,:] .- omega_ub[n,:] .+ psi_3[n] .* u .== 0)
    end

    for n in 1:N
        @constraint(model, pi_lb[n,:] .- pi_ub[n,:] .+ sum(Y[n,:,:] .* p_dag,dims=2) .== 0)
    end

    for n in 1:N
        @constraint(model, eta_lb[n,:] .- eta_ub[n,:] .- phi_3[n] .* u .== 0)
    end

    for n in 1:N
        @constraint(model, ups_lb[n,:] .- ups_ub[n,:] .- sum(Z[n,:,:] .* p_dag,dims=2) .== 0)
    end

    @constraint(model, sum(omega_ub .* A_ub) - sum(omega_lb .* A_lb) + sum(pi_ub .* B_ub) - sum(pi_lb .* B_lb) + delta + sum(psi_2) + sum(phi_1) <= 0)

    for n in 1:N
        @constraint(model, eta_ub[n,:]' * A_ub[n,:] - eta_lb[n,:]' * A_lb[n,:] + ups_ub[n,:]' * B_ub[n,:] - ups_lb[n,:]' * B_lb[n,:] + delta + psi_1[n] + phi_2[n] - X[n,:]' * p_dag[n,:] <= 0)
    end

    for n in 1:N
        @constraint(model, [psi_3[n], psi_2[n], psi_1[n]] in MOI.DualExponentialCone())
    end

    for n in 1:N
        @constraint(model, [phi_3[n],phi_2[n],phi_1[n]] in MOI.ExponentialCone())
    end

    for n in 1:N
        for j in 1:N
            for k in 1:K
                @constraint(model, Y[n,j,k] >= psi_lb * X[j,k])
                @constraint(model, Y[n,j,k] <= psi_ub * X[j,k])
                @constraint(model, Y[n,j,k] >= psi_3[n] - psi_ub * (1 - X[j,k]))
                @constraint(model, Y[n,j,k] <= psi_3[n] - psi_lb * (1 - X[j,k]))
            end
        end
    end

    for n in 1:N
        for j in 1:N
            for k in 1:K
                @constraint(model, Z[n,j,k] >= phi_lb * X[j,k])
                @constraint(model, Z[n,j,k] <= phi_ub * X[j,k])
                @constraint(model, Z[n,j,k] >= phi_3[n] - phi_ub * (1 - X[j,k]))
                @constraint(model, Z[n,j,k] <= phi_3[n] - phi_lb * (1 - X[j,k]))
            end
        end
    end

    @constraint(model, sum(X,dims=2) .== 1)

    for n in 1:N
        for j in 1:N
            @constraint(model, sum(Y[n,j,:]) == psi_3[n])
            @constraint(model, sum(Z[n,j,:]) == phi_3[n])
        end
    end


    @objective(model, Max, delta)

    optimize!(model)
    status = JuMP.termination_status(model)
    # println("status: ", status)
    # solution_summary(model)
    if status == MOI.OPTIMAL
        obj_val = objective_value(model)
        X_val = round.(value.(X))
        solve_time = JuMP.solve_time(model)
    else
        obj_val = NaN
        X_val = ones(N,N) .* NaN
        solve_time = NaN
    end
    return obj_val,X_val,solve_time
end

In [ ]:
function Solve_RO_Price_with_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,u,p_dag,psi_lb,psi_ub)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, delta)                           # 标量 δ
    @variable(model, omega_lb[1:N,1:N_u] >= 0)            
    @variable(model, omega_ub[1:N,1:N_u] >= 0)           
    @variable(model, pi_lb[1:N,1:N] >= 0)     
    @variable(model, pi_ub[1:N,1:N] >= 0)     

    # exponential variables
    @variable(model, psi_1[1:N])                   
    @variable(model, psi_2[1:N])                   
    @variable(model, psi_3[1:N])        
    
    @variable(model, X[1:N, 1:K], Bin)        # 二进制变量 x_{jk}
    @variable(model, Y[1:N,1:N,1:K] <= 0)    

    for n in 1:N
        @constraint(model, omega_lb[n,:] .- omega_ub[n,:] .+ psi_3[n] .* u .== 0)
    end

    for n in 1:N
        @constraint(model, pi_lb[n,:] .- pi_ub[n,:] .+ sum(Y[n,:,:] .* p_dag,dims=2) .== 0)
    end

    @constraint(model, sum(omega_ub .* A_ub) - sum(omega_lb .* A_lb) + sum(pi_ub .* B_ub) - sum(pi_lb .* B_lb) + delta + sum(psi_2) <= 0)

    for n in 1:N
        @constraint(model, delta + psi_1[n] - X[n,:]' * p_dag[n,:] <= 0)
    end

    for n in 1:N
        @constraint(model, [psi_3[n], psi_2[n], psi_1[n]] in MOI.DualExponentialCone())
    end


    # for n in 1:N
    #     for j in 1:N
    #         for k in 1:K
    #             @constraint(model, Y[n,j,k] >= psi_lb * X[j,k])
    #             @constraint(model, Y[n,j,k] <= psi_ub * X[j,k])
    #             @constraint(model, Y[n,j,k] >= psi_3[n] - psi_ub * (1 - X[j,k]))
    #             @constraint(model, Y[n,j,k] <= psi_3[n] - psi_lb * (1 - X[j,k]))
    #         end
    #     end
    # end

    for n in 1:N
        for j in 1:N
            for k in 1:K
                @constraint(model, Y[n,j,k] >= psi_lb * X[j,k])
                @constraint(model, Y[n,j,k] <= 0.0)
            end
        end
    end    
    for n in 1:N
        for j in 1:N
            @constraint(model, sum(Y[n,j,:]) == psi_3[n])
        end
    end

    @constraint(model, sum(X,dims=2) .== 1)
    @objective(model, Max, delta)

    optimize!(model)
    status = JuMP.termination_status(model)
    # println("status: ", status)
    # solution_summary(model)
    if status == MOI.OPTIMAL
        obj_val = objective_value(model)
        X_val = round.(value.(X))
        solve_time = JuMP.solve_time(model)
    else
        obj_val = NaN
        X_val = ones(N,N) .* NaN
        solve_time = NaN
    end
    return obj_val,X_val,solve_time
end

## Evaluate

### Parameters

In [ ]:
Random.seed!(1)
# 设置参数
N = 5   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2
offdiag_sign = "negative"
# 预设数据（根据你的实际问题替换这些值）
u = rand(N_u)        # u 向量 ∈ ℝ^N
A_true = rand(N,N_u)      # a_n ∈ ℝ^N
# B = rand(N,N)        # b_n ∈ ℝ^N
p_dag = round.(rand(N, K),digits=2)   # p_{nk}^† 数据矩阵
B_true = generate_strictly_row_diagonally_dominant(N, 1.0,offdiag_sign)

In [ ]:
bd_coef = 0.00001
A_lb = A_true .- bd_coef .* abs.(rand(N,N_u))
A_ub = A_true .+ bd_coef .* abs.(rand(N,N_u))
B_lb = B_true .- bd_coef .* abs.(rand(N,N))
B_ub = B_true .+ bd_coef .* abs.(rand(N,N))
# obj_RO,X_RO = Solve_RO(N,N_u,K,tau,A_lb,A_ub,B_lb,B_ub,u,p_dag);
# total_rev = compute_oof(X_RO, A, B, u, p_dag)
# println("total rev: ", total_rev) 
# price_RO = sum(X_RO .* p_dag,dims=2)
# println("price_RO: ", price_RO) 

### ETO

In [ ]:
obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_true,B_true,u,p_dag)
rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, u, p_dag)
println("rev_ETO=",rev_ETO,",price_ETO = ",price_ETO)

### RO with one-side exponential cone

In [ ]:
for i in 1:9
    psi_lb = -1.0 - i * 1.0
    psi_ub = 0.1
    # println("psi_lb = ",psi_lb,",phi_lb = ",phi_lb)
    obj_RO,X_RO,time_RO = Solve_RO_Price_with_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,u,p_dag,psi_lb,psi_ub)
    rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, u, p_dag)
    println("psi_lb = ",psi_lb,",rev_RO = ",rev_RO,",price_RO = ",price_RO)
end

In [ ]:
p_dag

### RO with two-side exponential cone

In [ ]:
for i in 1:9
    psi_lb = -0.33 - i * 0.001
    psi_ub = 0.1
    phi_lb = -0.33 - i * 0.001
    phi_ub = 0.1
    # println("psi_lb = ",psi_lb,",phi_lb = ",phi_lb)
    obj_RO,X_RO,time_RO = Solve_RO_Price(N,N_u,K,A_lb,A_ub,B_lb,B_ub,u,p_dag,psi_lb,psi_ub,phi_lb,phi_ub)
    rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, u, p_dag)
    println("psi_lb = ",psi_lb,",phi_lb = ",phi_lb,",rev_RO = ",rev_RO,",price_RO = ",price_RO)
end

### More experiments

In [ ]:
Random.seed!(1)
# 设置参数
N = 3   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2

offdiag_sign = "mix"
iterations = 10
bias_coef = [0.01,0.05,0.1]
ro_coef_all = [0.00001,0.01,0.1,0.15,0.2]

rev_True = zeros(iterations)

rev_ETO = zeros(iterations)
price_ETO = zeros(iterations,N)

rev_RO = zeros(iterations,length(ro_coef_all))
for iter in 1:iterations
    println("Iteration: ", iter)
    # 预设数据（根据你的实际问题替换这些值）
    u = rand(N_u)        # u 向量 ∈ ℝ^N
    A_true = rand(N,N_u)      # a_n ∈ ℝ^N
    # B = rand(N,N)        # b_n ∈ ℝ^N
    # p_dag = round.(rand(N, K),digits=2)  # p_{nk}^† 数据矩阵
    p_dag = [n * k for n in 1:N, k in 1:K]
    B_true = generate_strictly_row_diagonally_dominant(N, 1.0,offdiag_sign)

    obj_True,X_True,time_True = Solve_ETO(N,N_u,K,A_true,B_true,u,p_dag)
    rev_True[iter], price_True = compute_oof(X_True, A_true, B_true, u, p_dag)


    A_bias = randn(N,N_u)
    B_bias = generate_strictly_row_diagonally_dominant(N, 1.0,offdiag_sign)
    A_hat = A_true .+ 0.3 .* A_bias
    B_hat = B_true .- 0.3.* B_bias
    obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_hat,B_hat,u,p_dag)
    rev_ETO[iter], price_ETO[iter,:] = compute_oof(X_ETO, A_true, B_true, u, p_dag)

    ro_index = 1
    for ro_coef in ro_coef_all
        A_lb = A_hat .- ro_coef .* abs.(A_hat)
        A_ub = A_hat .+ ro_coef .* abs.(A_hat)
        B_lb = B_hat .- ro_coef .* abs.(B_hat)
        B_ub = B_hat .+ ro_coef .* abs.(B_hat)
        
        psi_lb = -10.0
        psi_ub = 0.1
        obj_RO,X_RO,time_RO = Solve_RO_Price_with_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,u,p_dag,psi_lb,psi_ub)
        rev_RO[iter,ro_index], price_RO = compute_oof(X_RO, A_true, B_true, u, p_dag)
        ro_index += 1
    end
end

In [ ]:
println("rev_True = ",mean(rev_True))
println("rev_ETO = ",mean(rev_ETO))
println("rev_RO = ",mean(rev_RO,dims=1))

In [ ]:
rev_ETO

In [ ]:
rev_RO